# 機械学習スクラッチ入門
### 【問題1】train_test_splitのスクラッチ

In [1]:
#基本ライブラリの読み込み
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
%matplotlib inline

In [2]:
def scratch_train_test_split(X, y, train_size=0.8,):
    """
    検証用データを分割する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定

    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      学習データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    #引数がPandasの場合、ndarrayへ変換
    if type(X) == pd.DataFrame:
        X = X.values
    if type(y) == pd.Series:
        y = y.values
    
    Xy = np.concatenate([X,y.reshape(y.shape[0],1)], axis=1)
    Xy = np.random.permutation(Xy)              #シャッフル
    sep_line = round(Xy.shape[0]*train_size)   #端数処理
    
    X_train = Xy[0:sep_line, 0:X.shape[1]]
    X_test = Xy[sep_line:, 0:X.shape[1]]
    y_train = Xy[0:sep_line, -1]
    y_test = Xy[sep_line: , -1]

    return X_train, X_test, y_train, y_test

#関数動作チェック
X = np.arange(20).reshape(10,2)
y = np.arange(10).reshape(10,1)
scratch_train_test_split(X,y)

(array([[14, 15],
        [ 8,  9],
        [18, 19],
        [ 4,  5],
        [16, 17],
        [ 0,  1],
        [10, 11],
        [ 6,  7]]), array([[ 2,  3],
        [12, 13]]), array([7, 4, 9, 2, 8, 0, 5, 3]), array([1, 6]))

### 【問題2】 分類問題を解くコードの作成

【問題１】で作成した　scratch_train_test_split　を使用して、通常の分類問題に挑む。

モデルは、
- ロジスティック回帰（SGDClassifierクラス）
  - 勾配降下法を用いて計算するモデル。引数でloss="log"とすることでロジスティック回帰の計算になる。
- SVM
- 決定木

の３種類使用し、また、データセットは以下の３種類となる。

- Iris
  - 2値分類としたいため、virgicolorとvirginicaの2つの目的変数のみ利用する。特徴量は4種類全て使う。
- DICオリジナル１
- DICオリジナル２

最終的に、３x３＝9回の分析を行う。

In [3]:
#ライブラリ
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

### Irisデータセット
---

In [4]:
#Irisデータセット
from sklearn.datasets import load_iris
iris = load_iris()
nd_Xy  = np.concatenate([iris.data, iris.target.reshape(150,1)], axis=1)

df = pd.DataFrame(nd_Xy, columns=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
df = df[df["species"].isin([1, 2])]

X_train, X_test, y_train, y_test = scratch_train_test_split(df.iloc[:, 0:4], df.iloc[:, -1])


In [5]:
#標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [7]:
#ロジスティク回帰
sgd = linear_model.SGDClassifier(loss="log", max_iter=1000, tol=1e-3)
sgd.fit(X_train , y_train)

#Recall値
print(metrics.recall_score(y_test, sgd.predict(X_test)))


1.0


In [8]:
#SVC
svc = SVC()
svc.fit(X_train , y_train)

#Recall値
print(metrics.recall_score(y_test, svc.predict(X_test)))


1.0


In [9]:
#DicisionTree
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

#Recall値
print(metrics.recall_score(y_test, dtc.predict(X_test)))

0.9090909090909091


### DICオリジナル１
---


In [10]:
#DICオリジナル１
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]

f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))

X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)

random_index = np.random.permutation(np.arange(n_samples))
X1 = X[random_index]
y1 = y[random_index]

X_train, X_test, y_train, y_test = scratch_train_test_split(X1, y1)

In [11]:
#標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [12]:
#ロジスティク回帰
sgd = linear_model.SGDClassifier(loss="log", max_iter=1000, tol=1e-3)
sgd.fit(X_train , y_train)

#Recall値
print(metrics.recall_score(y_test, sgd.predict(X_test)))

1.0


In [13]:
#SVC
svc = SVC()
svc.fit(X_train , y_train)

#Recall値
print(metrics.recall_score(y_test, svc.predict(X_test)))

1.0


In [14]:
#DicisionTree
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

#Recall値
print(metrics.recall_score(y_test, dtc.predict(X_test)))

1.0


### DICオリジナル２
---

In [15]:
#DICオリジナル２
X2 = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
       [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
       [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
       [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
       [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
       [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
       [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
       [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
       [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
       [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
       [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
       [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
       [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
       [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
       [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
       [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
       [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
       [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
       [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
       [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y2 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

X_train, X_test, y_train, y_test = scratch_train_test_split(X2, y2)

In [16]:
#標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [17]:
#ロジスティク回帰
sgd = linear_model.SGDClassifier(loss="log", max_iter=1000, tol=1e-3)
sgd.fit(X_train , y_train)

#Recall値
print(metrics.recall_score(y_test, sgd.predict(X_test)))

0.6


In [18]:
#SVC
svc = SVC()
svc.fit(X_train , y_train)

#Recall値
print(metrics.recall_score(y_test, svc.predict(X_test)))

0.6


In [19]:
#DicisionTree
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

#Recall値
print(metrics.recall_score(y_test, dtc.predict(X_test)))

1.0


### 【問題2】 回帰問題を解くコードの作成

線形回帰は勾配降下法を用いて計算するSGDRegressorクラスを利用して、House Pricesを分析する。

train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使う。


In [20]:
#データフレームにデータを格納
df_0 = pd.read_csv('train.csv', index_col=0)
df = df_0.loc[:,["GrLivArea", "YearBuilt", "SalePrice"]]
df.head()

,GrLivArea,YearBuilt,SalePrice
Id,,,
1,1710,2003,208500
2,1262,1976,181500
3,1786,2001,223500
4,1717,1915,140000
5,2198,2000,250000


In [21]:
#データ分割
X_train, X_test, y_train, y_test = scratch_train_test_split(df.iloc[:, 0:3], df.iloc[:, -1])

In [22]:
#標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

/Users/ishizucat/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [23]:
#ロジスティク回帰
sgdr = linear_model.SGDRegressor()
sgdr.fit(X_train , y_train)

/Users/ishizucat/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [24]:
#平均二乗誤差による評価
from sklearn.metrics import mean_squared_error

y_train_pred = sgdr.predict(X_train) # 学習データに対する目的変数を予測
y_pred = sgdr.predict(X_test)   # 検証データに対する目的変数を予測

print('MSE train data: ', mean_squared_error(y_train, y_train_pred)) # 学習データを用いたときの平均二乗誤差を出力
print('MSE test data : ', mean_squared_error(y_test, y_pred))         # 検証データを用いたときの平均二乗誤差を出力

MSE train data:  13831691.553991057
MSE test data :  11474491.519661328


以上